In [1]:
from scipy.stats import norm
from botorch.models import PairwiseLaplaceMarginalLogLikelihood
import time
import numpy.testing as npt
import torch
from botorch.fit import fit_gpytorch_mll
from sklearn.datasets import make_classification
from aepsych.models.ax_pairwise_probit import PairwiseGPModel

In [2]:
X, y = make_classification(
            n_samples=20,
            n_features=2,
            n_redundant=0,
            n_informative=1,
            random_state=1,
            n_clusters_per_class=1,
        )
x, y = torch.Tensor(X), torch.Tensor(y)
y = y.reshape(-1, 1)

# print("X: ", x, " y: ", y)

pairwise_model = PairwiseGPModel
pairwise_model.dim = x.shape[1]
pairwise_model.lb = torch.min(x, dim=0)[0]
pairwise_model.ub = torch.max(x, dim=0)[0]
pairwise_model = pairwise_model(None)

datapoints, comparisons = pairwise_model._pairs_to_comparisons(x, y)

# pairwise_model.likelihood = PairwiseProbitLikelihood()

# print("Datapoints: ", datapoints, " comparisons: ", comparisons)
pairwise_model.set_train_data(datapoints, comparisons)

mll = PairwiseLaplaceMarginalLogLikelihood(pairwise_model.likelihood, pairwise_model)
print("Starting fit...")
starttime = time.time()
fit_gpytorch_mll(mll)
print(f"Fit done, time={time.time()-starttime}")

# print("x5: ... 0 ", x[5:, ..., 0], " x5: ... 1 ", x[5:, ..., 1])
with torch.no_grad():
    f0, _ = pairwise_model.predict(x[5:], rereference=None)
    # f1, _ = pairwise_model.predict(x[5:, ..., 1])
    pred_diff = norm.cdf(f0)
pred = pred_diff > 0.5

# convert pred to 0/1 from numpy True/False
pred = pred.astype(int)

#convert pred to tensor
pred = torch.Tensor(pred)


print("pred: ", pred, " y[5:]: ", y[5:].reshape(-1))

npt.assert_allclose(pred.reshape(-1, 1), y[5:])

Starting fit...
Fit done, time=1.4314656257629395
pred:  tensor([1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1.])  y[5:]:  tensor([1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1.])
